In [38]:
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
from collections import defaultdict

ModuleNotFoundError: No module named 'matplotlib.backends.registry'

In [22]:
# Function to initialize the reward matrix R
def initial_R(A, Z, weight, A_Z_dict):
    R = defaultdict(lambda: defaultdict(float))
    for i in range(len(A)):
        R[A[i]][Z[i]] = weight[i]
    return R

# Function to initialize the Q-matrix
def initial_Q(R):
    Q = defaultdict(lambda: defaultdict(float))
    for state in R:
        for action in R[state]:
            Q[state][action] = 0.0
    return Q

# Function to update the Q-value for a state-action pair
def update_Q(T, Q, current_state, next_state, alpha):
    current_t = T[current_state][next_state]
    current_q = Q[current_state][next_state]
    new_q = current_q + alpha * (current_t + min(Q[next_state].values()) - current_q)
    Q[current_state][next_state] = new_q   
    return Q

In [24]:
def get_dict(data):
    # Ensure column names are stripped of whitespace
    data.columns = data.columns.str.strip()
    
    # Check if required columns are present
    required_columns = {"original", "connected", "weight"}
    if not required_columns.issubset(data.columns):
        raise KeyError(f"Missing required columns in CSV. Expected {required_columns}, but got {set(data.columns)}.")
    
    graph = {}
    graph["A"] = data["original"].tolist()  # Map 'original' to 'A'
    graph["Z"] = data["connected"].tolist()  # Map 'connected' to 'Z'
    graph["weight"] = data["weight"].tolist()
    
    # Create adjacency dictionary
    A_Z_dict = defaultdict(list)
    for a, z in zip(graph["A"], graph["Z"]):
        A_Z_dict[a].append(z)
    graph["A_Z_dict"] = A_Z_dict
    return graph


In [36]:
import matplotlib
print(matplotlib.__version__)

ModuleNotFoundError: No module named 'matplotlib.backends.registry'

In [26]:
# Q-learning function to find the optimal route
def get_result(R, Q, alpha, epsilon, n_episodes, start, end):
    for episode in range(n_episodes):
        current_state = start
        while current_state not in end:
            # Choose an action using epsilon-greedy policy
            if np.random.uniform(0, 1) < epsilon:
                # Explore: Random action
                possible_actions = list(R[current_state].keys())
                next_state = np.random.choice(possible_actions)
            else:
                # Exploit: Choose action with minimum Q-value
                next_state = min(Q[current_state], key=Q[current_state].get)
            
            # Update Q-values
            Q = update_Q(R, Q, current_state, next_state, alpha)
            current_state = next_state

    # Extract all routes and their Q-values
    all_routes = []
    for state in Q:
        for action in Q[state]:
            all_routes.append((state, action, Q[state][action]))
    
    return {"Q": Q, "all_routes": sorted(all_routes, key=lambda x: x[2])}

In [28]:
# Load the graph data (use either 'graph_1.csv' or 'graph_2.csv')
data = pd.read_csv("graph_1.csv")  # Change file name as needed
graph = get_dict(data)

# Extract graph components
A = graph["A"]
Z = graph["Z"]
weight = graph["weight"]
A_Z_dict = graph["A_Z_dict"]

print("Graph loaded successfully!")

Graph loaded successfully!
